<a href="https://colab.research.google.com/github/shivanii135/Task2/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5956 sha256=d87595d7397690fc0a023e03a696f483831635bef086c819693bc831e8f2a6c4
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [3]:
import pandas as pd
import numpy as np
from apyori import apriori

In [6]:
df = pd.read_csv("/content/Market_Basket_Optimisation.csv",header=None)

In [8]:
df.fillna(0,inplace=True)

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# Data Pre-processing step

# for using aprori , need to convert data in list format..
# transaction = [['apple','almonds'],['apple'],['banana','apple']]....

transactions = []

for i in range(0,len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0,20) if str(df.values[i,j])!='0'])

In [11]:
## verifying - by printing the 0th transaction
transactions[0]

['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil']

In [12]:
## verifying - by printing the 1st transaction
transactions[1]

['burgers', 'meatballs', 'eggs']

In [13]:
# Call apriori function which requires minimum support, confidance and lift, min length is combination of item default is 2".
rules = apriori(transactions, min_support=0.003, min_confidance=0.2, min_lift=3, min_length=2)

## min_support = 0.003 -> means selecting items with min support of 0.3%
## min_confidance = 0.2 -> means min confidance of 20%
## min_lift = 3
## min_length = 2 -> means no. of items in the transaction should be 2

In [14]:
#it generates a set of rules in a generator file...
rules

<generator object apriori at 0x7f2fdb7d5230>

In [ ]:
# all rules need to be converted in a list..
Results = list(rules)
Results

In [16]:
# convert result in a dataframe for further operation...
df_results = pd.DataFrame(Results)

In [17]:
# as we see "order_statistics" , is itself a list so need to be converted in proper format..
df_results.head()

,items,support,ordered_statistics
0,"(cottage cheese, brownies)",0.003466,"[((brownies), (cottage cheese), 0.102766798418..."
1,"(light cream, chicken)",0.004533,"[((chicken), (light cream), 0.0755555555555555..."
2,"(mushroom cream sauce, escalope)",0.005733,"[((escalope), (mushroom cream sauce), 0.072268..."
3,"(pasta, escalope)",0.005866,"[((escalope), (pasta), 0.07394957983193277, 4...."
4,"(fresh bread, tomato juice)",0.004266,"[((fresh bread), (tomato juice), 0.09907120743..."


In [18]:
# keep support in a separate data frame so we can use later..
support = df_results.support

In [19]:
first_values = []
second_values = []
third_values = []
fourth_value = []

# loop number of rows time and append 1 by 1 value in a separate list..
# first and second element was frozenset which need to be converted in list..
for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [20]:
# convert all four list into dataframe for further operation..
lhs = pd.DataFrame(first_values)
rhs = pd.DataFrame(second_values)

confidance=pd.DataFrame(third_values,columns=['Confidance'])

lift=pd.DataFrame(fourth_value,columns=['lift'])

In [21]:
# concat all list together in a single dataframe
df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final

,0,1,0,1,2,support,Confidance,lift
0,brownies,None,cottage cheese,None,None,0.003466,0.102767,3.225330
1,chicken,None,light cream,None,None,0.004533,0.075556,4.843951
2,escalope,None,mushroom cream sauce,None,None,0.005733,0.072269,3.790833
3,escalope,None,pasta,None,None,0.005866,0.073950,4.700812
4,fresh bread,None,tomato juice,None,None,0.004266,0.099071,3.259356
...,...,...,...,...,...,...,...,...
89,ground beef,pancakes,mineral water,spaghetti,None,0.003066,0.211009,3.532991
90,ground beef,None,tomatoes,mineral water,spaghetti,0.003066,0.031208,3.344117
91,olive oil,None,mineral water,milk,spaghetti,0.003333,0.050607,3.216994
92,mineral water,milk,shrimp,spaghetti,None,0.003066,0.063889,3.014029


In [22]:
df_final.fillna(value=' ', inplace=True)
df_final.head()

,0,1,0,1,2,support,Confidance,lift
0,brownies,,cottage cheese,,,0.003466,0.102767,3.225330
1,chicken,,light cream,,,0.004533,0.075556,4.843951
2,escalope,,mushroom cream sauce,,,0.005733,0.072269,3.790833
3,escalope,,pasta,,,0.005866,0.073950,4.700812
4,fresh bread,,tomato juice,,,0.004266,0.099071,3.259356


In [23]:
#set column name
df_final.columns = ['lhs',1,'rhs',2,3,'support','confidance','lift']
df_final.head()

,lhs,1,rhs,2,3,support,confidance,lift
0,brownies,,cottage cheese,,,0.003466,0.102767,3.225330
1,chicken,,light cream,,,0.004533,0.075556,4.843951
2,escalope,,mushroom cream sauce,,,0.005733,0.072269,3.790833
3,escalope,,pasta,,,0.005866,0.073950,4.700812
4,fresh bread,,tomato juice,,,0.004266,0.099071,3.259356


In [24]:
df_final.drop(columns=[1,2,3],inplace=True)

In [25]:
df_final.head()

,lhs,rhs,support,confidance,lift
0,brownies,cottage cheese,0.003466,0.102767,3.225330
1,chicken,light cream,0.004533,0.075556,4.843951
2,escalope,mushroom cream sauce,0.005733,0.072269,3.790833
3,escalope,pasta,0.005866,0.073950,4.700812
4,fresh bread,tomato juice,0.004266,0.099071,3.259356


In [26]:
## Showing top 10 items, based on lift.  Sorting in desc order
df_final.sort_values('lift', ascending=False).head(10)

,lhs,rhs,support,confidance,lift
58,olive oil,mineral water,0.003866,0.058704,6.115863
6,fromage blanc,honey,0.003333,0.245098,5.164271
49,ground beef,tomato sauce,0.003066,0.031208,4.980600
1,chicken,light cream,0.004533,0.075556,4.843951
3,escalope,pasta,0.005866,0.073950,4.700812
28,ground beef,french fries,0.003200,0.032564,4.697422
11,pasta,shrimp,0.005066,0.322034,4.506672
23,ground beef,chocolate,0.003999,0.040706,4.490183
69,frozen vegetables,shrimp,0.003200,0.033566,4.417225
10,olive oil,whole wheat pasta,0.007999,0.121457,4.122410
